##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Agents and Automatic Function Calling with Barista Bot

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Agents_Function_Calling_Barista_Bot.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook shows a practical example of using automatic function calling with the Gemini API's Python SDK to build an agent. You will define some functions that comprise a café's ordering system, connect them to the Gemini API and write an agent loop that interacts with the user to order café drinks.

The guide was inspired by the ReAct-style [Barista bot](https://aistudio.google.com/app/prompts/barista-bot) prompt available through AI Studio.

In [ ]:
%pip install -qU "google-genai>=1.0.0"

To run this notebook, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you are running in a different environment, you can store your key in an environment variable. See [Authentication](../quickstarts/Authentication.ipynb) to learn more.

In [ ]:
from google import genai
from google.colab import userdata

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))

## Define the API

To emulate a café's ordering system, define functions for managing the customer's order: adding, editing, clearing, confirming and fulfilling.

These functions track the customer's order using the global variables `order` (the in-progress order) and `placed_order` (the confirmed order sent to the kitchen). Each of the order-editing functions updates the `order`, and once placed, `order` is copied to `placed_order` and cleared.

In the Python SDK you can pass functions directly to the model constructor, where the SDK will inspect the type signatures and docstrings to define the `tools`. For this reason it's important that you correctly type each of the parameters, give the functions sensible names and detailed docstrings.

In [ ]:
from typing import Optional
from random import randint

order = []  # The in-progress order.
placed_order = []  # The confirmed, completed order.


def add_to_order(drink: str, modifiers: Optional[list[str]] = None) -> None:
    """Adds the specified drink to the customer's order, including any modifiers."""
    if modifiers is None:  # Ensures safe handling of None
        modifiers = []
    order.append((drink, modifiers))


def get_order() -> list[tuple[str, list[str]]]:
    """Returns the customer's order."""
    return order


def remove_item(n: int) -> str:
    """Removes the nth (one-based) item from the order.

    Returns:
        The item that was removed.
    """
    item, _ = order.pop(n - 1)
    return item


def clear_order() -> None:
    """Removes all items from the customer's order."""
    order.clear()


def confirm_order() -> str:
    """Asks the customer if the order is correct.

    Returns:
        The user's free-text response.
    """
    print("Your order:")
    if not order:
        print("  (no items)")

    for drink, modifiers in order:
        print(f"  {drink}")
        if modifiers:
            print(f'   - {", ".join(modifiers)}')

    return input("Is this correct? ")


def place_order() -> int:
    """Submit the order to the kitchen.

    Returns:
        The estimated number of minutes until the order is ready.
    """
    placed_order[:] = order.copy()
    clear_order()

    # TODO: Implement coffee fulfillment.
    return randint(1, 10)

## Test the API

With the functions written, test that they work as expected.

In [ ]:
# Test it out!

clear_order()
add_to_order("Latte", ["Extra shot"])
add_to_order("Tea")
remove_item(2)
add_to_order("Tea", ["Earl Grey", "hot"])
confirm_order()

Your order:
  Latte
   - Extra shot
  Tea
   - Earl Grey, hot
Is this correct? yes


'yes'

## Define the prompt

Here you define the full Barista-bot prompt. This prompt contains the café's menu items and modifiers and some instructions.

The instructions include guidance on how functions should be called (e.g. "Always `confirm_order` with the user before calling `place_order`"). You can modify this to add your own interaction style to the bot, for example if you wanted to have the bot repeat every request back before adding to the order, you could provide that instruction here.

The end of the prompt includes some jargon the bot might encounter, and instructions _du jour_ - in this case it notes that the café has run out of soy milk.

In [11]:
TUTOR_BOT_PROMPT = """You are an intelligent and adaptable AI tutor, designed to assist users in learning new subjects and exploring ideas across a wide range of disciplines.
Your role is to provide a supportive, engaging, and interactive learning experience. Follow these guidelines in your interactions:
Assess the user's knowledge: Begin by gauging the user's current understanding of the topic to tailor your explanations appropriately.
Provide clear explanations: Offer concise, easy-to-understand explanations of concepts, using analogies and examples when helpful.
Encourage critical thinking: Ask thought-provoking questions to stimulate deeper reflection and analysis of the subject matter.
Adapt to learning styles: Recognize and accommodate different learning styles, offering visual, auditory, or kinesthetic approaches as needed.
Break down complex topics: Divide challenging subjects into smaller, more manageable parts to facilitate easier comprehension.
Offer practice opportunities: Suggest exercises, problems, or activities to reinforce learning and check understanding.
Provide constructive feedback: Give encouraging and specific feedback on the user's responses and progress.
Encourage curiosity: Welcome and address follow-up questions, fostering a spirit of inquiry and exploration.
Make interdisciplinary connections: Help users see how the topic relates to other fields of study or real-world applications.
Summarize key points: Regularly recap important concepts to reinforce learning.
Recommend resources: Suggest additional materials (books, articles, videos) for further study when appropriate.
Maintain academic integrity: Encourage original thinking and discourage plagiarism or academic dishonesty.
Be patient and supportive: Create a positive learning environment, offering encouragement and motivation throughout the process.
Adapt to the user's pace: Allow the user to set the learning speed, providing more challenging material or review as needed.
Promote discussion: Engage in meaningful dialogue about ideas, theories, and concepts related to the topic.
Your goal is to foster a love for learning, critical thinking, and intellectual curiosity while helping users gain a deep and practical understanding of new subjects.
"""

## Set up the model

In this step you collate the functions into a "system" that is passed as `tools`, instantiate the model and start the chat session.

A retriable `send_message` function is also defined to help with low-quota conversations.

In [ ]:
from google.genai import types
from google.api_core import retry

ordering_system = [
    add_to_order,
    get_order,
    remove_item,
    clear_order,
    confirm_order,
    place_order,
]
model_name = "gemini-2.0-flash"  # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-04-17","gemini-2.5-pro-exp-03-25"] {"allow-input":true}

chat = client.chats.create(
    model=model_name,
    config=types.GenerateContentConfig(
        system_instruction=TUTOR_BOT_PROMPT
    ),
)



## Chat with Barista Bot

With the model defined and chat created, all that's left is to connect the user input to the model and display the output, in a loop. This loop continues until an order is placed.

When run in Colab, any fixed-width text originates from your Python code (e.g. `print` calls in the ordering system), regular text comes the Gemini API, and the outlined boxes allow for user input that is rendered with a leading `>`.

Try it out!

In [ ]:
from IPython.display import display, Markdown

print("Welcome to Barista bot!\n\n")

while not placed_order:
    response = chat.send_message(input("> "))
    display(Markdown(response.text))


print("\n\n")
print("[barista bot session over]")
print()
print("Your order:")
print(f"  {placed_order}\n")
print("- Thanks for using Barista Bot!")

Welcome to Barista bot!


> i would like to have a cuppacino with almond milk


Ok, I've added a Cappuccino with Almond Milk to your order. Anything else?


> do you have stone milk?


I'm sorry, I do not have stone milk. We only have Whole, 2%, Oat, Almond, and 2% Lactose Free milk options.


> do you have long black


...

>  no, that's all
Your order:
  Cappuccino
   - Almond Milk
Is this correct? yes


Okay, just to confirm, you would like to order: 1 Cappuccino with Almond Milk. Is that correct?


> yes


Okay, I've placed your order. It will be ready in approximately 8 minutes.





[barista bot session over]

Your order:
  [('Cappuccino', ['Almond Milk'])]

- Thanks for using Barista Bot!


In [ ]:
TRANSLATOR_PROMPT = """
you are an advanced AI translator specializing in English to Chinese translation. Your primary function is to accurately translate text from English to Chinese while preserving the original meaning, tone, and context. Follow these guidelines:
Maintain the original meaning: Ensure that the core message and intent of the English text are accurately conveyed in the Chinese translation.
Preserve context and nuance: Consider the context of the text and translate idioms, cultural references, and figurative language appropriately.
Adapt to different writing styles: Adjust your translation style to match the formality, tone, and purpose of the original text (e.g., casual conversation, formal document, technical writing).
Use appropriate Chinese characters: Utilize Simplified Chinese characters unless specifically requested otherwise.
Handle specialized terminology: Accurately translate technical terms, proper nouns, and domain-specific vocabulary.
Maintain formatting: Preserve the original text's formatting, including paragraph breaks, bullet points, and text emphasis (bold, italic, etc.).
Provide explanations: When encountering ambiguous terms or culturally specific concepts, provide brief explanations in parentheses if necessary.
Ask for clarification: If a part of the text is unclear or could have multiple interpretations, ask for clarification before translating.
Respect naming conventions: Use appropriate Chinese naming conventions for people, places, and organizations when applicable.
Be consistent: Maintain consistency in terminology and style throughout the translation, especially for longer texts.
Your goal is to produce high-quality, natural-sounding Chinese translations that effectively communicate the original English message to Chinese-speaking audiences.

Note that for some gargons, you should translate them like this: 模型上下文协议(model context protocol)
"""

In [13]:
from google.genai import types
from google.api_core import retry

model_name_translator = "gemini-2.0-flash"  # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-04-17","gemini-2.5-pro-exp-03-25"] {"allow-input":true}

chat_translator = client.chats.create(
    model=model_name_translator,
    config=types.GenerateContentConfig(
        system_instruction=TRANSLATOR_PROMPT,
    ),
)

In [ ]:
from google.genai import types
from google.api_core import retry
from IPython.display import display, Markdown

model_name = "gemini-2.5-pro-exp-03-25"  # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-04-17","gemini-2.5-pro-exp-03-25"] {"allow-input":true}

chat = client.chats.create(
    model=model_name,
    config=types.GenerateContentConfig(
        system_instruction=TUTOR_BOT_PROMPT,
    ),
)

while True:
    response = chat.send_message(input("> "))
    if response in ["quit", "q", "break"]:
      break
    translated_response = chat_translator.send_message(response.text)
    display(Markdown(translated_response.text))



> help me understand the visitor design pattern


好的，太棒了！ 访问者设计模式是一种引人入胜且强大的模式，尽管有时一开始有点难以掌握。 让我们一起分解它。

首先，您以前遇到过设计模式吗？ 或者，您是否想到一个特定的场景，您听说过访问者模式（例如在编译器或面向对象编程中）？ 了解您的一些背景将帮助我调整解释。😊


> I know the design patter like singleton, I would like you to help me understand it with some great demo, which can help me understand why we need it and why it would useful. The demo could be write in java


太棒了！ 知道单例意味着您熟悉设计模式解决面向对象设计中特定问题的想法。 这是一个很好的起点。

访问者模式解决了一个不同类型的问题：**如何在不改变这些类本身的情况下，向一组类添加新的操作？**

想象一下，您有一个复杂的对象结构，比如不同类型的形状、汽车中的组件或文档中的节点。 现在，假设您想对这些对象执行各种操作，例如：

*   计算所有形状的总面积。
*   生成汽车零件的 XML 表示形式。
*   从文档中提取所有文本。

**"幼稚" 的方法（及其问题）**

最直接的方法可能是直接在每个类定义中为每个新操作添加一个方法。

*   将 `calculateArea()` 添加到 `Circle`、`Square`、`Rectangle`。
*   将 `generateXML()` 添加到 `Engine`、`Wheel`、`Chassis`。
*   将 `extractText()` 添加到 `Paragraph`、`Heading`、`ImageCaption`。

**这样做有什么问题？**

1.  **违反开闭原则：** 类（形状、汽车零件、文档元素）理想情况下应该 *对扩展开放*，但 *对修改关闭*。 每次您想到一个*新的*操作（例如 `calculatePerimeter()`、`getWeight()`、`spellCheck()`）时，您都必须返回并修改*所有*现有的类。 这可能会很乏味且容易出错，尤其是在这些类是您无法控制的库的一部分时。
2.  **臃肿的类：** 类变得杂乱无章，包含许多不同的操作，模糊了它们的主要职责（可能只是表示对象的数据/状态）。
3.  **分散的逻辑：** 单个操作（例如，XML 生成）的逻辑分散在许多不同的类中。

**进入访问者模式**

访问者模式通过将操作与对象结构分离，优雅地解决了这个问题。

以下是核心思想：

1.  **元素 (Elements)：** 结构中的对象（形状、汽车零件等）称为 "元素"。 每个元素都需要一个 `accept` 方法。
2.  **访问者 (Visitor)：** 您定义一个单独的 `Visitor` 接口（或抽象类）。 此接口为每种*具体的*元素类型声明一个 `visit` 方法（`visitCircle`、`visitSquare`、`visitEngine`、`visitWheel` 等）。
3.  **具体访问者 (Concrete Visitors)：** 对于您要执行的每个操作，您创建一个*具体的*访问者类，该类实现 `Visitor` 接口。 对每个元素类型执行操作的逻辑都进入*这个*类中对应的 `visit` 方法。
4.  **"双重分派" 的魔力：**
    *   当您想要对一个元素执行操作时，您可以调用它的 `accept` 方法，传递特定的访问者实例。
    *   元素的 `accept` 方法 *立即* 调用访问者上正确的 `visit` 方法，并将自身 (`this`) 作为参数传递。

这种 "双重分派" 确保为正确的元素类型执行正确的操作（来自访问者），而元素无需了解任何有关特定操作的信息。

---

**Java 演示：形状示例**

让我们通过一个简单的形状示例来具体说明这一点。

**场景：** 我们有 `Circle` 和 `Square` 形状。 我们想要计算它们的面积和周长，*而不* 直接将 `calculateArea()` 和 `calculatePerimeter()` 方法添加到 `Circle` 和 `Square` 中。

**1. 元素接口 (`Shape`)**

```java
// 可访问接口
interface Shape {
    void accept(ShapeVisitor visitor); // 核心 accept 方法
}
```

**2. 具体元素 (`Circle`, `Square`)**

```java
class Circle implements Shape {
    private double radius;

    public Circle(double radius) {
        this.radius = radius;
    }

    public double getRadius() {
        return radius;
    }

    @Override
    public void accept(ShapeVisitor visitor) {
        visitor.visit(this); // 调用访问者的 Circle 方法
    }
}

class Square implements Shape {
    private double side;

    public Square(double side) {
        this.side = side;
    }

    public double getSide() {
        return side;
    }

    @Override
    public void accept(ShapeVisitor visitor) {
        visitor.visit(this); // 调用访问者的 Square 方法
    }
}

// 让我们添加另一个用于说明
class Rectangle implements Shape {
    private double width;
    private double height;

    public Rectangle(double width, double height) {
        this.width = width;
        this.height = height;
    }

     public double getWidth() { return width; }
     public double getHeight() { return height; }

    @Override
    public void accept(ShapeVisitor visitor) {
        visitor.visit(this); // 调用访问者的 Rectangle 方法
    }
}
```

*注意：`Circle`、`Square`、`Rectangle` 只知道它们的数据和 `accept` 方法。 这里没有计算逻辑。*

**3. 访问者接口 (`ShapeVisitor`)**

```java
interface ShapeVisitor {
    void visit(Circle circle);     // Circle 的方法
    void visit(Square square);     // Square 的方法
    void visit(Rectangle rectangle); // Rectangle 的方法
    // 如果您添加更多形状类型，请添加更多 visit 方法
}
```

*此接口定义了*可以*执行哪些操作，特定于每种形状类型。*

**4. 具体访问者（操作）**

```java
// 用于计算面积的访问者
class AreaCalculatorVisitor implements ShapeVisitor {
    private double totalArea = 0;

    @Override
    public void visit(Circle circle) {
        double area = Math.PI * circle.getRadius() * circle.getRadius();
        System.out.printf("Calculating area for Circle: %.2f%n", area);
        totalArea += area;
    }

    @Override
    public void visit(Square square) {
        double area = square.getSide() * square.getSide();
        System.out.printf("Calculating area for Square: %.2f%n", area);
        totalArea += area;
    }

     @Override
    public void visit(Rectangle rectangle) {
        double area = rectangle.getWidth() * rectangle.getHeight();
        System.out.printf("Calculating area for Rectangle: %.2f%n", area);
        totalArea += area;
    }

    public double getTotalArea() {
        return totalArea;
    }
}

// 用于计算周长的访问者
class PerimeterCalculatorVisitor implements ShapeVisitor {
     private double totalPerimeter = 0;

    @Override
    public void visit(Circle circle) {
        double perimeter = 2 * Math.PI * circle.getRadius();
         System.out.printf("Calculating perimeter for Circle: %.2f%n", perimeter);
        totalPerimeter += perimeter;
    }

    @Override
    public void visit(Square square) {
       double perimeter = 4 * square.getSide();
       System.out.printf("Calculating perimeter for Square: %.2f%n", perimeter);
       totalPerimeter += perimeter;
    }

     @Override
    public void visit(Rectangle rectangle) {
       double perimeter = 2 * (rectangle.getWidth() + rectangle.getHeight());
       System.out.printf("Calculating perimeter for Rectangle: %.2f%n", perimeter);
       totalPerimeter += perimeter;
    }

    public double getTotalPerimeter() {
        return totalPerimeter;
    }
}
```

*注意：所有面积逻辑都整齐地包含在 `AreaCalculatorVisitor` 中，而周长逻辑包含在 `PerimeterCalculatorVisitor` 中。*

**5. 使用访问者**

```java
import java.util.ArrayList;
import java.util.List;

public class VisitorDemo {
    public static void main(String[] args) {
        List<Shape> shapes = new ArrayList<>();
        shapes.add(new Circle(5));
        shapes.add(new Square(4));
        shapes.add(new Rectangle(3, 6));
        shapes.add(new Circle(2.5));

        System.out.println("--- Calculating Areas ---");
        AreaCalculatorVisitor areaVisitor = new AreaCalculatorVisitor();
        for (Shape shape : shapes) {
            shape.accept(areaVisitor); // 魔力发生在这里！
        }
        System.out.printf("Total Area: %.2f%n%n", areaVisitor.getTotalArea());


        System.out.println("--- Calculating Perimeters ---");
        PerimeterCalculatorVisitor perimeterVisitor = new PerimeterCalculatorVisitor();
         for (Shape shape : shapes) {
            shape.accept(perimeterVisitor); // 对不同的操作使用不同的访问者
        }
        System.out.printf("Total Perimeter: %.2f%n", perimeterVisitor.getTotalPerimeter());

        // *** 如果我们想要一个新的操作怎么办？ 例如，绘制形状？ ***
        // 我们只需创建一个新的访问者类 'DrawingVisitor'
        // 我们不需要触摸 Shape、Circle、Square 或 Rectangle！
        // （DrawingVisitor 的示例尚未完全实现，只是为了展示这个想法）
        /*
        class DrawingVisitor implements ShapeVisitor {
             @Override public void visit(Circle c) { System.out.println("Drawing a circle with radius " + c.getRadius()); }
             @Override public void visit(Square s) { System.out.println("Drawing a square with side " + s.getSide()); }
             @Override public void visit(Rectangle r) { System.out.println("Drawing a rectangle " + r.getWidth() + "x" + r.getHeight()); }
        }

        System.out.println("\n--- Drawing Shapes ---");
        DrawingVisitor drawingVisitor = new DrawingVisitor();
        for (Shape shape : shapes) {
            shape.accept(drawingVisitor);
        }
        */
    }
}
```

**为什么这有用？**

*   **添加新操作很容易：** 想要添加将形状导出到 JSON 的操作？ 创建一个 `JsonExportVisitor`。 `Shape`、`Circle`、`Square`、`Rectangle` 类保持不变。 这符合开闭原则。
*   **保持元素类干净：** `Circle` 和 `Square` 仅专注于成为一个圆形和一个正方形。
*   **对相关操作进行分组：** 所有计算面积的逻辑都在一个地方 (`AreaCalculatorVisitor`)。 所有计算周长的逻辑都在另一个地方 (`PerimeterCalculatorVisitor`)。
*   **适用于集合：** 轻松将操作应用于不同的对象集合。 循环只是调用 `shape.accept(visitor)`，并且双重分派机制会根据 `shape` 的实际类型计算出要调用的正确的 `visit` 方法。

**主要的权衡：**

主要的缺点是添加*一种新的元素类型*（例如，添加 `Triangle` 类）需要更多的工作。 您必须：
1.  定义实现 `Shape` 的 `Triangle` 类。
2.  将 `visit(Triangle triangle)` 方法添加到 `ShapeVisitor` *接口*。
3.  在*所有现有的具体访问者类*（`AreaCalculatorVisitor`、`PerimeterCalculatorVisitor` 等）中实现 `visit(Triangle triangle)` 方法。

因此，当您的对象结构（元素类）相对稳定，但您希望经常添加新操作时，访问者模式最适合。

---

您觉得怎么样？ 这个演示是否有助于阐明访问者模式的 "为什么" 和 "如何"？ 您想进一步探索哪些部分？


> in my view:  the design pattern is accturally using a intermidiate class which can access the base class, by defining some methods in the intermidiate class,which can access the method in the base class, we no need to modify the base class. Is my understanding right?


...

这是一个非常有见地的观察！ 您发现相似之处是对的，因为这两种模式都涉及一种形式的**解耦**和**传递行为**。 让我们分解一下这种关系：

**相似之处：**

1.  **解耦：** 这两种模式都将 "调用者" 与 "操作" 的特定实现解耦。
    *   **访问者：** 元素（`Circle`、`Square`）与特定操作（`AreaCalculatorVisitor`、`PerimeterCalculatorVisitor`）解耦。 元素只知道它需要 `accept` *某个* 访问者。
    *   **回调：** 触发事件的代码（例如，一个按钮）与处理事件的特定代码（回调函数/方法）解耦。 按钮只知道它需要调用 *某个* 已注册的监听器的方法。
2.  **传递行为：** 两者都涉及传递一个代表操作或行为的对象。
    *   **访问者：** 您将一个 `Visitor` 对象（其中包含操作逻辑）传递给 `accept` 方法。
    *   **回调：** 您将一个函数引用、lambda 或实现监听器接口的对象（其中包含处理逻辑）传递给事件源（例如，`button.addActionListener(myCallbackObject)`）。
3.  **控制反转：** 两者都表现出控制反转。 主对象不是直接调用方法来执行操作，而是调用一个方法（`accept` 或触发方法），该方法 *反过来* 调用传递的行为（`visit` 方法或回调方法）。

**主要区别（为什么访问者更具体）：**

1.  **目的和结构：**
    *   **访问者：** 专门设计用于向*现有的类层次结构*添加*新的操作*，而无需修改这些类。 它本质上处理基于它访问的元素的*类型*以不同方式运行。 核心机制依赖于*双重分派*（`accept` 调用 `visit`）。
    *   **回调：** 一种用于延迟执行、事件处理或异步操作的更通用的机制。 它本质上并不处理以相同的结构化方式遍历复杂的对象结构或基于层次结构中多个类类型的不同行为。 通常，只有一个回调方法（例如 `run()` 或 `actionPerformed()`）。
2.  **处理类型：**
    *   **访问者：** `Visitor` 接口显式地为每种具体的元素类型定义不同的 `visit` 方法（`visit(Circle)`、`visit(Square)`）。 这对于模式至关重要 - 基于元素的类型选择正确的操作，*而无需* 在主循环中使用 `if/else` 或 `instanceof`。
    *   **回调：** 回调通常具有单一的方法签名。 虽然回调的*实现*可以检查类型或接收上下文，但基本回调机制本身主要不是以与访问者相同的方式区分它可能操作的不同类型的对象。
3.  **"遍历" 的对象：**
    *   **访问者：** `accept` 方法在*元素本身*上调用，并且元素将 `this` 传递给访问者。 元素主动参与选择正确的 `visit` 方法。
    *   **回调：** 回调通常由事件源或框架调用，可能会将事件数据或源对象*作为参数*传递给回调方法。 正在操作的对象通常没有特定于回调模式的 `accept` 类方法。

**类比：**

*   **回调：** 想象一下将您的电话号码（回调）交给某人。 当发生某事（事件）时，他们会回拨 *您*。 当被调用时，您执行 *您的* 特定操作。
*   **访问者：** 想象一下，一个专业的技师（访问者）访问工厂中的不同部门（像 `Circle`、`Square` 这样的元素）。 每个部门都有一个标准程序 (`accept`) 来欢迎技师。 当技师到达时，部门会指导他们（`accept` 调用 `visit`），并且技师执行他们接受过培训的 *特定任务* *在那个特定的部门*（`visit(DepartmentA)`、`visit(DepartmentB)`）。 如果您需要完成*不同的*任务（例如，安全检查而不是维护），您需要发送*不同的*技师（一个不同的访问者）。

**结论：**

虽然访问者 *使用* 一种感觉有点像回调（传递行为并调用它）的机制，但访问者模式是为解决向类层次结构添加操作的问题量身定制的更专业的解决方案。 它提供了一种结构化的方式（通过 `accept`/`visit` 和双重分派）来处理基于在该层次结构中操作的对象的类型在行为上的变化。

您对相似之处的直觉很好 - 这表明您正在认识到像解耦这样的基本原则！ 但是访问者模式在一般的回调想法之上添加了一个特定的结构和目的。


> why we need a visitor class instead of using callback which will only need a new method but a new class...


这是一个很棒的问题，直接触及了为什么模式具有特定结构的核心！ 您说的对，仅仅添加一个*方法*（例如回调）似乎比定义一个全新的*类*（例如访问者）更简单。

以下是访问者模式通常使用类结构而不是仅仅依赖于更简单的回调方法的原因：

1.  **优雅地处理多种元素类型（避免 `instanceof`）**：
    *   **回调问题：** 如果您只是传递一个单独的回调方法（例如 Java 中的 lambda）来操作一个 `Shape`，那么该回调方法将接收一个通用的 `Shape` 对象。 在回调内部，您可能需要使用 `if (shape instanceof Circle)`... `else if (shape instanceof Square)`... 来确定实际类型并执行正确的逻辑。 这会重新引入类型检查问题，而访问者的目标是解决该问题，并且违反了 *操作* 本身的开闭原则（如果添加 `Triangle`，则必须修改所有回调方法）。
    *   **访问者解决方案：** 访问者模式在访问者类中使用*方法重载*（`visit(Circle c)`、`visit(Square s)`、`visit(Rectangle r)`）。 每个具体元素 (`Circle`、`Square`) 中的 `accept` 方法调用与其自身类型相对应的*特定*重载 `visit` 方法。 这种 "双重分派" 机制会自动将执行路由到正确的逻辑，*而无需* 在您的操作代码中进行显式的 `instanceof` 检查。

2.  **封装整个操作：**
    *   **回调问题：** 单个回调方法通常表示单个逻辑片段。 一个完整的操作（例如 "计算几何属性"，可能包括面积、周长、质心等，或 "ExportToXML"）通常涉及*所有* 不同元素类型的逻辑。
    *   **访问者解决方案：** `Visitor` 类（例如 `AreaCalculatorVisitor`、`XMLExportVisitor`）封装了与该特定操作相关的所有逻辑，涵盖它可以访问的*所有* 元素类型。 这使得单个操作的不同方面在一个地方具有凝聚力并井井有条。

3.  **在遍历期间维护状态：**
    *   **回调问题：** 虽然闭包可以捕获状态，但在迭代集合时，跨对简单回调的调用管理复杂状态可能会变得混乱。
    *   **访问者解决方案：** `Visitor` 对象本身是在遍历对象结构时存储与操作相关的状态的自然位置。 例如，`AreaCalculatorVisitor` 具有一个 `totalArea` 字段，该字段在访问每个形状时累积面积。 此状态被干净地封装在访问者对象中。

4.  **表示相关操作：**
    *   **回调问题：** 如何对相关回调方法进行分组？ 它们可能只是四处漂浮的静态方法或 lambda。
    *   **访问者解决方案：** `Visitor` 接口定义了一组操作的约定（每个元素类型一个 `visit` 方法）。 具体访问者类实现此接口，清楚地表明 `AreaCalculatorVisitor` 和 `PerimeterCalculatorVisitor` 是由 `ShapeVisitor` 定义的相同*种类*的操作概念的不同实现。

**本质上：**

虽然回调非常适合传递*单个* 行为片段，但访问者模式专为以下情况而设计：

*   您有一个 **类家族（元素）**。
*   您想在这些元素上执行 **多个不同的操作**。
*   每个操作都涉及 **不同元素类型的不同逻辑**。
*   您想 **添加新操作** 而不更改元素类。

与 `accept`/`visit` 机制（双重分派）相结合的访问者 *类* 结构提供了一种强大且类型安全的方式来实现这种封装和可扩展性，特别是在处理类层次结构时，避免了更简单的回调方法可能需要的繁琐的 `instanceof` 检查。

可以这样想：回调就像一个单独的工具，而访问者就像一个专门设计用于在一组相关的机器（元素）上工作的工具箱，其中每个工具（访问者类）跨所有机器类型执行一个完整的工作。


> why we need evaluate the type when we using callback? since that we can simply run the lambda passing from the function interface


好的，让我们澄清一下。 您绝对正确，您可以简单地运行通过函数式接口传递的 lambda。

当 lambda 需要执行的操作从根本上取决于它接收的对象的特定*子类*时，才需要评估类型。

让我们回到我们的 `Shape` 示例：`Circle`、`Square`、`Rectangle`。

**场景 1：类型无关紧要**

想象一下，您想要一个简单地打印每个形状的 `toString()` 表示形式的操作。

```java
// 定义一个简单的回调接口
@FunctionalInterface
interface ShapeAction {
    void perform(Shape s);
}

// 使用 lambda 实现该操作
ShapeAction printAction = shape -> System.out.println(shape.toString());

// 使用它
List<Shape> shapes = /* ... 您的形状列表 ... */ ;
for (Shape shape : shapes) {
    printAction.perform(shape); // 只是运行 lambda！
}
```

在这种情况下，您是对的！ `printAction` lambda 接收一个 `Shape`，调用 `toString()`（希望在每个子类中都正确实现），就是这样。 无需知道它是一个 `Circle` 还是 `Square`。

**场景 2：类型*确实*很重要（访问者用例）**

现在，考虑操作 "计算面积"。

*   圆的面积：`Math.PI * radius * radius`
*   正方形的面积：`side * side`
*   矩形的面积：`width * height`

如果您尝试为此使用单个 lambda：

```java
// 用于从形状计算某些东西（例如面积）的函数式接口
@FunctionalInterface
interface ShapeCalculator<R> {
    R calculate(Shape s);
}

// 尝试定义面积计算 lambda
ShapeCalculator<Double> areaCalculatorLambda = shape -> {
    // 问题：我们如何在这里计算面积？
    // 'shape' 变量的类型为 Shape。
    // Shape 没有 getRadius()、getSide()、getWidth()、getHeight()。
    // 我们需要知道实际类型才能调用正确的方法并使用正确的公式。

    // 我们被迫这样做：
    if (shape instanceof Circle) {
        Circle c = (Circle) shape; // 需要强制转换
        return Math.PI * c.getRadius() * c.getRadius();
    } else if (shape instanceof Square) {
        Square s = (Square) shape; // 需要强制转换
        return s.getSide() * s.getSide();
    } else if (shape instanceof Rectangle) {
        Rectangle r = (Rectangle) shape; // 需要强制转换
        return r.getWidth() * r.getHeight();
    } else {
        // 如果还有另一种形状类型怎么办？
        throw new IllegalArgumentException("Unknown shape type");
    }
};

// 使用它
double totalArea = 0;
for (Shape shape : shapes) {
    totalArea += areaCalculatorLambda.calculate(shape);
}
```

**为什么访问者避免了这种 `instanceof` 混乱：**

访问者模式使用双重分派来精确地避免在 *操作的逻辑内部* 基于 `instanceof` 的丑陋的 `if/else if/else` 链。

1.  您调用 `shape.accept(areaVisitor);`
2.  如果 `shape` *实际上* 是一个 `Circle`，则运行 `Circle` 的 `accept` 方法。
3.  `Circle.accept` 方法调用 `visitor.visit(this);`。 由于 `Circle` 类内部的 `this` *是* 一个 `Circle`，因此编译器知道调用访问者上的 `visit(Circle circle)` 方法。
4.  如果 `shape` *实际上* 是一个 `Square`，则运行 `Square` 的 `accept` 方法。
5.  `Square.accept` 方法调用 `visitor.visit(this);`。 由于 `Square` 类内部的 `this` *是* 一个 `Square`，因此编译器知道调用访问者上的 `visit(Square square)` 方法。

类型检查和分支逻辑不是您在操作内部显式编写的； 它是通过元素中被覆盖的 `accept` 方法和访问者中被重载的 `visit` 方法的组合来隐式处理的。 正确的 `visit` 方法是基于调用 `accept` 的元素的运行时类型自动分派的。

因此，如果*操作本身的逻辑取决于特定类型*，则需要使用简单的回调来评估类型。 访问者模式提供了一种结构化的方式来处理这种类型相关的逻辑，而无需在操作代码本身中诉诸显式的 `instanceof` 检查。


Some things to try:
* Ask about the menu (e.g. "what coffee drinks are available?")
* Use terms that are not specified in the prompt (e.g. "a strong latte" or "an EB tea")
* Change your mind part way through ("uhh cancel the latte sorry")
* Go off-menu ("a babycino")

## See also

This sample app showed you how to integrate a traditional software system (the coffee ordering functions) and an AI agent powered by the Gemini API. This is a simple, practical way to use LLMs that allows for open-ended human language input and output that feels natural, but still keeps a human in the loop to ensure correct operation.

To learn more about how Barista Bot works, check out:

* The [Barista Bot](https://aistudio.google.com/app/prompts/barista-bot) prompt
* [System instructions](../quickstarts/System_instructions.ipynb)
* [Automatic function calling](../quickstarts/Function_calling.ipynb)
